<a href="https://colab.research.google.com/github/LeonardoGastaldo/labo2025v/blob/main/LG_Optimizacion_Bayesiana_para_Arboles_Azarosos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aca correr con Python

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi

Aca correr con R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Oct 24 09:14:08 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,664022,35.5,1454474,77.7,1454474,77.7
Vcells,1235912,9.5,8388608,64.0,1975128,15.1


In [ ]:
# Instalar librería (solo 1 vez)
install.packages("ParBayesianOptimization")

# Cargar paquetes
library(ParBayesianOptimization)
# cargo las librerias que necesito
require("data.table")
require("rpart")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: data.table

Loading required package: rpart



In [ ]:
setwd("/content/buckets/b1/exp")
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [ ]:
# Cargar datos y preparar entorno
set.seed(101581)
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]
campos_buenos <- setdiff(colnames(dtrain), "clase_ternaria")

# Función objetivo para evaluación
funcion_objetivo <- function(cp, minsplit, minbucket, maxdepth) {
  resultados <- c()

  for (i in 1:5) {
    campos_random <- sample(campos_buenos, size = as.integer(length(campos_buenos) * 0.5))
    formula <- as.formula(paste("clase_ternaria ~", paste(campos_random, collapse = " + ")))

    modelo <- rpart(
      formula = formula,
      data = dtrain,
      xval = 0,
      control = rpart.control(
        cp = cp,
        minsplit = as.integer(minsplit),
        minbucket = as.integer(minbucket),
        maxdepth = as.integer(maxdepth)
      )
    )

    # Aquí la métrica: suma de las probabilidades de "BAJA+2"
    pred <- predict(modelo, dfuture, type = "prob")[, "BAJA+2"]
    resultados <- c(resultados, sum(pred, na.rm = TRUE))
  }

  return(list(Score = mean(resultados)))
}

# Ejecutar Optimización Bayesiana
optimizacion <- bayesOpt(
  FUN = funcion_objetivo,
  bounds = list(
    cp = c(-0.5, -0.001),
    minsplit = c(500L, 5000L),
    minbucket = c(10L, 1000L),
    maxdepth = c(4L, 30L)
  ),
  initPoints = 10,
  iters.n = 20,
  acq = "ucb",
  kappa = 2.576,
  verbose = 1
)

# Mostrar mejores hiperparámetros
print(getBestPars(optimizacion))

# Mostrar resumen de todas las combinaciones evaluadas
print(scoreSummary(optimizacion))



Running initial scoring function 10 times in 1 thread(s)...$cp
[1] -0.001

$minsplit
[1] 5000

$minbucket
[1] 857

$maxdepth
[1] 26



ERROR: Error in scoreSummary(optimizacion): could not find function "scoreSummary"


In [ ]:
format(Sys.time(), "%a %b %d %X %Y")